pip install markitdown   
This is an open-source package for converting PDF to MarkDown.

In [4]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert("data/2024全球经济金融展望报告.pdf")
with open("data/md_file.md", "w", encoding="utf-8") as f:
    f.write(result.text_content)

Merge all page content into one string

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
import re

loader = PyPDFLoader("data/2024全球经济金融展望报告.pdf")
documents = loader.load()

### documents is a list where each element is a Document object, which respresenting each page of content loaded from  the PDF document.
pattern = r"^全球经济金融展望报告\n中国银行研究院 \d+ 2024年"
merged_docs = [Document(page_content='\n'.join(re.sub(pattern, '', doc.page_content) for doc in documents))]


split document

In [11]:
import os
import pickle
from uuid import uuid4
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        print('found cache, restoring...')
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs


In [13]:
splitted_docs = split_docs(documents, os.path.join("outputs", 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)
splitted_docs_large = split_docs(merged_docs, os.path.join("outputs", 'split_docs_large.pkl'), chunk_size=1500, chunk_overlap=100)

extract qa

In [15]:
qa_gen_prompt_tmpl = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成8个问题、用户回答这个问题的上下文，和基于上下文对问题的回答。

对问题、上下文、答案的要求：

问题要与这段文本相关，不要询问类似“这个问题的答案在哪一章”这样的问题
上下文：上下文必须与原始文本的内容保持一致，不要进行缩写、扩写、改写、摘要、替换词语等
答案：回答请保持完整且简洁，无须重复问题。答案要能够独立回答问题，而不是引用现有的章节、页码等

返回结果以JSON形式组织，格式为[{"question": "...", "context": ..., "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

qa_gen_prompt_tmpl_large_context = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成2个问题，和基于这段文本对问题的回答，回答请保持完整，无须重复问题。
尽可能创建一些需要综合*大段*文本才能回答的问题，但不要问类似“这一段主要讲了什么内容”这样的问题，答案要能够独立回答问题，而不是引用现有的章节、页码等；不要问具体过于细节的问题，例如“海湾国家的2024年预期经济增长率是多少”，而是尽可能问类似“2024年全球经济的几大趋势是什么”、“受局部中东地区紧张局势影响，可能对全球原物料有哪些影响”。
返回结果以JSON形式组织，格式为[{"question": "...", "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

In [32]:
from openai import OpenAI
import time
import random

from zhipuai import ZhipuAI
client = ZhipuAI(api_key="")  
# response = client.chat.completions.create(
#     model="glm-4-plus",  
#     messages=[
#         {"role": "system", "content": "你是一个乐于回答各种问题的小助手，你的任务是提供专业、准确、有洞察力的建议。"},
#         {"role": "user", "content": "你是谁"},
#     ],
#     stream=True,
# )
# for chunk in response:
#     print(chunk.choices[0].delta.content)

def build_qa_prompt(prompt_tmpl, text):
    prompt = prompt_tmpl.replace('{{document}}', text).strip()
    return prompt

def chat(prompt, max_retry=3, debug=False, top_p=0.95, temperature=0.85):
    def do_chat(prompt):
        completion = client.chat.completions.create(
            model="glm-4-plus",
            messages=[
                {"role": "system", "content": "你是一个乐于回答各种问题的小助手，你的任务是提供专业、准确、有洞察力的建议。"},
                {"role": "user", "content": prompt}
            ],
            top_p=top_p,
            temperature=temperature
        )
        return completion.choices[0].message.content

    while max_retry > 0:
        try:
            return do_chat(prompt)
        except Exception as e:
            max_retry -= 1
            sleep_seconds = random.randint(1, 4)
            if debug:
                print(f"{str(e)}, remain retry: {max_retry}, sleeping {sleep_seconds}s {prompt}")
            time.sleep(sleep_seconds)
    return None

In [35]:
text = splitted_docs[40].page_content
text

'全球经济金融展望报告\n中国银行研究院 40 2024年\n注重绿色经济发展，为中海经贸合作创造了良好机遇。\n（一）近年来海湾六国经济表现受能源价格影响较大\n受国际能源价格走势影响，近年来海湾六国经济增长波动较大。2022年，\n在全球油气价格飙升推动下，海湾六国经济增长强劲，实际GDP增速达7.3%。\n其中，沙特占据海湾六国经济体量的半壁江山。2022年，沙特GDP规模达1.1\n万亿美元，阿联酋和卡塔尔GDP规模分别为5075亿和2373亿美元，位列第二\n和第三位（图26）。油气部门在海湾六国经济总量中的占比超过40%，在沙特\n的占比更是接近七成。\n图26：2022年海湾六国经济体量（亿美元）\n资料来源：世界银行，中国银行研究院\n2023年以来，海湾六国经济增长明显放缓。主要经济体货币政策加速收紧、\n全球经济活动减少使全球能源需求下降，海湾六国经济增长面临严重挑战。IMF\n在最新展望报告中将海湾六国2023年经济增速下调0.8个百分点至1.7%。其中，\n沙特经济增速预计仅为0.8%，低于中东石油出口国的平均水平。相比之下，阿\n联酋韧性较强，预计2023年和2024年经济增速分别为3.4%和4.0%，为海湾\n六国中最高（图27）。鉴于海湾六国经济结构相对单一，非石油部门虽然为海'

In [36]:
print(chat(build_qa_prompt(qa_gen_prompt_tmpl, text), debug=True))

```json
[
    {
        "question": "近年来海湾六国经济增长波动的主要原因是什么？",
        "context": "受国际能源价格走势影响，近年来海湾六国经济增长波动较大。",
        "answer": "受国际能源价格走势影响。"
    },
    {
        "question": "2022年海湾六国实际GDP增速是多少？",
        "context": "2022年，在全球油气价格飙升推动下，海湾六国经济增长强劲，实际GDP增速达7.3%。",
        "answer": "7.3%。"
    },
    {
        "question": "2022年沙特GDP规模是多少？",
        "context": "2022年，沙特GDP规模达1.1万亿美元。",
        "answer": "1.1万亿美元。"
    },
    {
        "question": "2022年阿联酋和卡塔尔的GDP规模分别是多少？",
        "context": "2022年，阿联酋和卡塔尔GDP规模分别为5075亿和2373亿美元，位列第二和第三位（图26）。",
        "answer": "阿联酋5075亿美元，卡塔尔2373亿美元。"
    },
    {
        "question": "油气部门在海湾六国经济总量中的占比是多少？",
        "context": "油气部门在海湾六国经济总量中的占比超过40%，在沙特的占比更是接近七成。",
        "answer": "超过40%。"
    },
    {
        "question": "2023年海湾六国经济增长明显放缓的原因是什么？",
        "context": "2023年以来，海湾六国经济增长明显放缓。主要经济体货币政策加速收紧、全球经济活动减少使全球能源需求下降，海湾六国经济增长面临严重挑战。",
        "answer": "主要经济体货币政策加速收紧、全球经济活动减少使全球能源需求下降。"
    },
    {
        "question": "IMF将海湾六国2023年经济增速下调